# Analyzing CIA Data

This project utilizes SQL to analyze a database from CIA World Factbook, a compendium of statistics about all of the countries on Earth. We'll be looking at population statistics and sorting through a database.

In [7]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db #This connects our notebook to the file

# First Glance

In [37]:
%%sql
SELECT *
  FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


It looks like these are the columns we'll be working with:
- name : The name of the country
- area : The country's total area (both land and water)
- area_land : Country's land area in square km
- area_water : Country's water area in square km
- population : Country's population
- population_growth : Country's population growth as a %
- birth_rate : Country's birth rate, or the number of births per year per 1,000 people.
- death_rate : Country's death rate, or the number of deaths per year per 1,000 people. 

# Min/Maxes

Let's just explore some of the basic functions of SQL by identifying the minimums and maximums in terms of population and population growth.

In [9]:
%%sql
SELECT MIN(population) AS 'Minimum Population',
       MAX(population) AS 'Maximum Population',
        Min(population_growth) AS 'Minimum Population Growth',
        Max(population_growth) AS 'Maximum Population Growth'
    FROM facts;

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


There is a country with 0 people and one with 7 billion, which is probably just the world. And also for growth there is a country with 0 growth and the highest being 4.02. 

# Identification 

Let's identify which countries exhibit these rates and populations by doing some nested queries.

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population) 
                         FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Antartica is the country with no one living there as of now.

In [11]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population) 
                         FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


And for some reason the world is counted as a country in this dataset, a little cheap for my taste but I guess it's a cool statistic to look at. 

# Removing the Whole World

Although this chapter title sounds dramatic, we'll simply be removing the world row from previous calculations to get rid of it's huge outlier. 

In [38]:
%%sql
SELECT MIN(population) AS 'Minimum Population',
       MAX(population) AS 'Maximum Population',
        MIN(population_growth) AS 'Minimum Population Growth',
        MAX(population_growth) AS 'Maximum Population Growth'
    FROM facts
    WHERE name <> 'World' 

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


With the removal of world, we see a dramatic drop in maximum population as that was obviously the placeholder for that statistics.

In [16]:
%%sql
SELECT ROUND(AVG(population),2) AS 'Average Population',
       ROUND(AVG(area),2) AS 'Average area of Country'
    FROM facts
    WHERE name <> 'World';

Done.


Average Population,Average area of Country
32242666.57,555093.55


We see that the average population is around 32 million and the average area is 555 thousand square kilometers

# Densely populated Areas

To finish we'll write a query that matches the following:
- Above-average values for population.
- Below-average values for area.

In [36]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                       FROM facts
                       WHERE name <> 'World') AND
    area < (SELECT AVG(area) 
           FROM facts
           WHERE name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Countries in this list are well-known to be small but populous, so this result does make sense. 